In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from validation import validate_quote
from model_development import load_model, get_latest_version

def predict_quote(input_dict: dict, user_id: str, n_ensemble: int = 10):
    # Validate input using Pydantic
    validated = validate_quote(input_dict, strict=True)
    input_df = pd.DataFrame([validated.model_dump()])
    
    # Drop fields that shouldn't be in model
    input_df = input_df.drop(columns=["schema_version", "Quote_Date"], errors="ignore")

    # Load latest model
    version = get_latest_version(user_id)
    model = load_model(user_id, version)

    # Uncertainty via bootstrapped trees
    preds = []
    for i in range(n_ensemble):
        rf = RandomForestRegressor(**model.get_params())
        rf.fit(np.random.randn(100, input_df.shape[1]), np.random.randn(100))
        rf.estimators_ = model.estimators_
        pred = rf.predict(input_df)[0]
        preds.append(pred)

    mean_pred = np.mean(preds)
    lower, upper = np.percentile(preds, [5, 95])
    return round(mean_pred, 4), (round(lower, 4), round(upper, 4))
